In [18]:
from pulp import *
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB


In [19]:
items = ['Handbag', 'Travel Bag', 'Wallet']
profit = {
    'Handbag': 60,
    'Travel Bag': 76,
    'Wallet': 30,
}
rm_mat1 = {
    'Handbag': 8,
    'Travel Bag': 10,
    'Wallet': 6,

}
rm_mat2 = {
    'Handbag': 16,
    'Travel Bag': 22,
    'Wallet': 10,

}
rm_mat3 = {
    'Handbag': 3.5,
    'Travel Bag': 4.5,
    'Wallet': 2.3,

}

prb = LpProblem('maximize_profit', LpMaximize)
item_vars = LpVariable.dicts('items', items, 0)
item_vars
prb += lpSum(profit[i]*item_vars[i] for i in items)
prb += lpSum([rm_mat1[i]*item_vars[i] for i in items]) <= 360
prb += lpSum([rm_mat2[i]*item_vars[i] for i in items]) <= 150
prb += lpSum([rm_mat3[i]*item_vars[i] for i in items]) <= 400
prb.writeLP('maximize_profit.lp')
prb.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/dt/hgtf7kbn7kv7f3gbs3wm7bs80000gn/T/7e3562a38b7b43c6b9b81775396d0dcf-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/dt/hgtf7kbn7kv7f3gbs3wm7bs80000gn/T/7e3562a38b7b43c6b9b81775396d0dcf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 21 RHS
At line 25 BOUNDS
At line 26 ENDATA
Problem MODEL has 3 rows, 3 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-1) rows, 3 (0) columns and 6 (-3) elements
0  Obj -0 Dual inf 201 (3)
1  Obj 562.5
Optimal - objective value 562.5
After Postsolve, objective 562.5, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 562.5 - 1 iterations time 0.002, Presolve 0.00
Option for pri

1

In [20]:
print("_________________________________________________________________________________")
print(f'Status: {LpStatus[prb.status]}')


for i in prb.variables():
    print(f'{i.name} : {i.varValue} per unit')
    print("_________________________________________________________________________________")
print(f'Maximum daily profits {value(prb.objective)}')
print("_________________________________________________________________________________")

_________________________________________________________________________________
Status: Optimal
items_Handbag : 9.375 per unit
_________________________________________________________________________________
items_Travel_Bag : 0.0 per unit
_________________________________________________________________________________
items_Wallet : 0.0 per unit
_________________________________________________________________________________
Maximum daily profits 562.5
_________________________________________________________________________________


In [21]:

retailers, salesPoints, luxbagsMarket = gp.multidict({
    (1): [11, 34],
    (2): [47, 411],
    (3): [44, 82],
    (4): [25, 157],
    (5): [10, 5],
    (6): [26, 183],
    (7): [26, 14],
    (8): [54, 215],
    (9): [18, 102],
    (10): [51, 21],
    (11): [20, 54],
    (12): [105, 0],
    (13): [7, 6],
    (14): [16, 96],
    (15): [34, 118],
    (16): [100, 112],
    (17): [50, 535],
    (18): [21, 8],
})


intl_retailers,  lbagMarket1 = gp.multidict({
    (1): 9,
    (2): 13,
    (3): 14,
    (4): 17,
    (5): 18,
    (6): 19,
    (7): 23,
    (8): 21
})


dom_retailers,  lbagMarket2 = gp.multidict({
    (9): 9,
    (10): 11,
    (11): 17,
    (12): 18,
    (13): 18,
    (14): 17,
    (15): 22,
    (16): 24,
    (17): 36,
    (18): 43
})


groupA,  retailerA = gp.multidict({
    (1): 1,
    (2): 1,
    (3): 1,
    (5): 1,
    (6): 1,
    (10): 1,
    (15): 1,
})


groupB,  retailerB = gp.multidict({
    (4): 1,
    (7): 1,
    (8): 1,
    (9): 1,
    (11): 1,
    (12): 1,
    (13): 1,
    (14): 1,
    (16): 1,
    (17): 1,
    (18): 1,
})


salesPoints40 = 300
salesPoints5 = 37.5
luxbagsMarket40 = 958
luxbagsMarket5 = 119.75
lbagMarket1_40 = 68.6
lbagMarket1_5 = 6.7
lbagMarket2_40 = 86
lbagMarket2_5 = 23.75
retailerA40 = 3.2
retailerA5 = 5.5
retailerB40 = 6
retailerB5 = 0.75
model = gp.Model('MarketAllocation')

allocate = model.addVars(retailers, vtype=GRB.BINARY, name="allocate")


salesPointsBop = model.addVar(ub=salesPoints5, name='salesPointsBop')
salesPointsEop = model.addVar(ub=salesPoints5, name='salesPointsEop')


luxbagsMarketBop = model.addVar(ub=luxbagsMarket5, name='luxBagsMarketBop')
luxbagsMarketEop = model.addVar(ub=luxbagsMarket5, name='luxBagsMarketEop')


lbagMarket1Bop = model.addVar(ub=lbagMarket1_5, name='lbagMarket1Bop')
lbagMarket1Eop = model.addVar(ub=lbagMarket1_5, name='lbagMarket1Eop')


lbagMarket2Bop = model.addVar(ub=lbagMarket2_5, name='lbagMarket2Bop')
lbagMarket2Eop = model.addVar(ub=lbagMarket2_5, name='lbagMarket2Eop')


retailerABop = model.addVar(ub=retailerA5, name='retailerABop')
retailerAEop = model.addVar(ub=retailerA5, name='retailerAEop')


retailerBBop = model.addVar(ub=retailerB5, name='retailerBPop')
retailerBEop = model.addVar(ub=retailerB5, name='retailerBEop')
SVConstr = model.addConstr((gp.quicksum(salesPoints[r]*allocate[r] for r in retailers)
                            + salesPointsBop - salesPointsEop == salesPoints40), name='SVConstrs')
lmConstr = model.addConstr((gp.quicksum(luxbagsMarket[r]*allocate[r] for r in retailers)
                            + luxbagsMarketBop - luxbagsMarketEop == luxbagsMarket40), name='lmConstr')
LB1Constr = model.addConstr((gp.quicksum(lbagMarket1[r]*allocate[r] for r in intl_retailers)
                            + lbagMarket1Bop - lbagMarket1Eop == lbagMarket1_40), name='LB1Constr')
LB2Constr = model.addConstr((gp.quicksum(lbagMarket2[r]*allocate[r] for r in dom_retailers)
                            + lbagMarket2Bop - lbagMarket2Eop == lbagMarket2_40), name='LB2Constr')
AConstr = model.addConstr((gp.quicksum(retailerA[r]*allocate[r] for r in groupA)
                           + retailerABop - retailerAEop == retailerA40), name='AConstr')
BConstr = model.addConstr((gp.quicksum(retailerB[r]*allocate[r] for r in groupB)
                           + retailerBBop - retailerBEop == retailerB40), name='BConstr')
obj = salesPointsBop + salesPointsEop + luxbagsMarketBop + luxbagsMarketEop + lbagMarket1Bop + \
    lbagMarket1Eop + lbagMarket2Bop + lbagMarket2Eop + \
    retailerABop + retailerAEop + retailerBBop + retailerBEop






Set parameter Username


In [22]:
model.setObjective(obj)
model.write('marketAllocation.lp')
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 30 columns and 83 nonzeros
Model fingerprint: 0xc82b7a72
Variable types: 12 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [8e-01, 1e+02]
  RHS range        [3e+00, 1e+03]
Found heuristic solution: objective 200.3000000
Presolve time: 0.00s
Presolved: 6 rows, 30 columns, 83 nonzeros
Variable types: 12 continuous, 18 integer (18 binary)

Root relaxation: objective 0.000000e+00, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    6  200.30000    0.00000   100%     -    0s
H    0     0                     145.4000000    0.00000   100%     -    0s
H    0 

In [23]:
print("\n\n_________________________________________________________________________________")
print(f"The optimal allocation of Luxury Leather Bags to retailers is:")
print("_________________________________________________________________________________")
for r in retailers:
    if(allocate[r].x > 0.5):
        print(f"Retailer{r}")
print("_________________________________________________________________________________")



_________________________________________________________________________________
The optimal allocation of Luxury Leather Bags to retailers is:
_________________________________________________________________________________
Retailer2
Retailer4
Retailer5
Retailer7
Retailer9
Retailer11
Retailer12
Retailer14
Retailer15
_________________________________________________________________________________
